In [ ]:
# Azure storage access info for open datasets NYC yellow taxi
storage_account_name = "azureopendatastorage"
container_name = "nyctlc"

sas_token = r"" # Specify blank since container is public with anonymous access

# Set Spark configuration to access  blob storage
spark.conf.set("fs.azure.sas.%s.%s.blob.core.windows.net" % (container_name, storage_account_name),sas_token)

directory = "yellow"
year = 2016
months = "1,2,3,4,5,6"
wasbs_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{directory}"
nyc_yellowtaxi_df = spark.read.parquet(wasbs_path)

# Filter data by year 2016 and months 1-6
filtered_nyc_yellowtaxi_df = nyc_yellowtaxi_df.filter(f"puYear = {year} AND puMonth IN ({months})")

In [ ]:
table_name = "nyc_yellowtaxi_raw"
filtered_nyc_yellowtaxi_df.write.mode("overwrite").format("delta").saveAsTable(f"{table_name}")
print(f"Spark dataframe (filtered_nyc_yellowtaxi_df) saved to a delta table: {table_name}")

In [ ]:
%%sql
select puYear, puMonth, count(*) from nyc_yellowtaxi_raw group by puYear, puMonth order by puYear, puMonth

In [ ]:
filtered_nyc_yellowtaxi_df.count()